 <!-- # Qwen3 + Qwen Embedding + Elasticsearch + Qdrant + LangGraph 토이 RAG 파이프라인


- LLM: **Qwen3-8B-Instruct** (로컬 LLM, LM Studio에서 OpenAI 호환 서버로 실행)
- Embedding: **Qwen2.5-Embedding** (Hugging Face Transformers)
- Sparse Search: **Elasticsearch (BM25)**
- Dense Search: **Qdrant + Qwen2.5-Embedding**
- Orchestration: **LangGraph StateGraph**
- 시나리오: *간단한 논문들(더미 데이터)을 ES/Qdrant에 넣고 → LangGraph 에이전트로 검색 + 요약*
 -->


<!-- ## 0. 사전 준비 (노트북 밖에서 할 일)

### 0-1. Docker로 ES + Qdrant 실행 (추천)

`docker-compose.yml` 예시 (로컬 프로젝트 루트에 저장):

```yaml
version: "3.8"

services:
  es:
    image: docker.elastic.co/elasticsearch/elasticsearch:8.14.0
    container_name: paper_search_es
    environment:
      - discovery.type=single-node
      - xpack.security.enabled=false
      - ES_JAVA_OPTS=-Xms1g -Xmx1g
    ports:
      - "9200:9200"

  qdrant:
    image: qdrant/qdrant:latest
    container_name: paper_search_qdrant
    ports:
      - "6333:6333"
```

터미널에서:
여러 개의 Docker 컨테이너(서비스)를 백그라운드에서 실행
도커 실행 = 서버 하나를 기동해서 그 안에서 Elasticsearch 서버가 돌아가게 한다는 의미
```bash
docker compose up -d
```

---

### 0-2. LM Studio에서 Qwen3-8B-Instruct 실행

1. LM Studio 설치
2. 모델 브라우저에서 `Qwen3-8B-Instruct` (또는 비슷한 이름의 GGUF 4bit) 다운로드
3. Server 모드로 실행 (OpenAI 호환 API 모드 켜기)
   - 기본 주소 예시: `http://localhost:1234/v1`
   - 모델 이름은 LM Studio UI에서 확인 (예: `qwen3-8b-instruct`)

 **LM Studio 서버를 OpenAI 호환 API**처럼 `requests`로 직접 호출하는 방식 -->


In [15]:
# !pip install -q langgraph elasticsearch qdrant-client transformers accelerate sentencepiece requests typing-extensions

<!-- ## 2. 공통 import 및 스키마 정의 -->

In [16]:
from __future__ import annotations

from dataclasses import dataclass
from enum import Enum
from typing import List, Optional, Dict, Any
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, END

from elasticsearch import Elasticsearch, helpers
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

from transformers import AutoTokenizer, AutoModel
import torch
import math
import requests
import json


In [17]:
class UtteranceType(str, Enum):
    KEYWORD_TOPIC = "keyword_topic"         # 키워드 기반 주제 질의
    NL_TOPIC = "natural_language_topic"     # 자연어 기반 주제 질의
    SPECIFIC_PAPER = "specific_paper"       # 특정 논문 지목형


@dataclass
class SearchStrategy:
    utterance_type: UtteranceType
    use_keyword_search: bool
    use_semantic_abstract_search: bool
    use_chunk_search: bool
    keyword_weight: float
    semantic_weight: float
    relevance_weight: float
    recency_weight: float
    citation_weight: float


@dataclass
class Paper:
    paper_id: str
    title: str
    abstract: str
    year: Optional[int] = None
    venue: Optional[str] = None
    citations: Optional[int] = None
    url: Optional[str] = None


@dataclass
class PaperHit:
    paper: Paper
    score: float


@dataclass
class ChunkHit:
    paper_id: str
    section: Optional[str]
    chunk_id: str
    text: str
    score: float


class AgentState(TypedDict, total=False):
    # 입력
    query_text: str

    # 분류/전략
    utterance_type: str
    strategy: SearchStrategy

    # 검색 결과
    keyword_hits: List[PaperHit]
    semantic_hits: List[PaperHit]
    merged_hits: List[PaperHit]
    top_papers: List[PaperHit]
    chunks: List[ChunkHit]

    # 기타 의존성
    lm_client: Any

    # 최종 출력
    answer: Optional[str]

<!-- ## 3. 발화 유형별 SearchStrategy 설정 -->

In [18]:
SEARCH_STRATEGY_MAP: Dict[UtteranceType, SearchStrategy] = {
    UtteranceType.KEYWORD_TOPIC: SearchStrategy(
        utterance_type=UtteranceType.KEYWORD_TOPIC,
        use_keyword_search=True,
        use_semantic_abstract_search=True,
        use_chunk_search=False,
        keyword_weight=0.7,
        semantic_weight=0.3,
        relevance_weight=0.6,
        recency_weight=0.2,
        citation_weight=0.2,
    ),
    UtteranceType.NL_TOPIC: SearchStrategy(
        utterance_type=UtteranceType.NL_TOPIC,
        use_keyword_search=True,
        use_semantic_abstract_search=True,
        use_chunk_search=True,
        keyword_weight=0.4,
        semantic_weight=0.6,
        relevance_weight=0.7,
        recency_weight=0.15,
        citation_weight=0.15,
    ),
    UtteranceType.SPECIFIC_PAPER: SearchStrategy(
        utterance_type=UtteranceType.SPECIFIC_PAPER,
        use_keyword_search=True,
        use_semantic_abstract_search=False,
        use_chunk_search=True,
        keyword_weight=1.0,
        semantic_weight=0.0,
        relevance_weight=0.9,
        recency_weight=0.05,
        citation_weight=0.05,
    ),
}

<!-- ## 4. ES / Qdrant / Embedding / LLM 클라이언트 설정 -->

In [19]:
# Elasticsearch 클라이언트 (Docker 기준 localhost:9200)
es = Elasticsearch(hosts=["http://localhost:9200"])

# Qdrant 클라이언트 (Docker 기준 localhost:6333)
qdrant = QdrantClient(host="localhost", port=6333)

# Qwen2.5 Embedding 모델 (처음 실행 시 다운로드 → 시간이 조금 걸릴 수 있음)
EMBED_MODEL_NAME = "BAAI/bge-m3"  # 필요 시 변경 가능


class QwenEmbedding:
    def __init__(self, model_name: str = EMBED_MODEL_NAME, device: str = None):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.model.eval()
        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = device
        self.model.to(self.device)
        # 차원 수는 모델마다 다르지만, Qwen2.5-Embedding은 일반적으로 1024 또는 1536 근처
        # 여기서는 runtime에서 확인해본다.
        dummy = self.encode(["hello"])
        self.dim = dummy.shape[1]

    def encode(self, texts: List[str]) -> Any:
        with torch.no_grad():
            inputs = self.tokenizer(
                texts,
                padding=True,
                truncation=True,
                return_tensors="pt",
            ).to(self.device)
            outputs = self.model(**inputs)
            # 간단히 mean pooling
            embeddings = outputs.last_hidden_state.mean(dim=1)
            return embeddings.cpu().numpy()


embedder = QwenEmbedding()

In [20]:
# LM Studio OpenAI 호환 서버용 간단 LLM 클라이언트
class LMStudioClient:
    def __init__(self, base_url: str = "http://localhost:1234/v1", model: str = "qwen3-8b-instruct"):
        self.base_url = base_url.rstrip("/")
        self.model = model

    def chat(self, system_prompt: str, user_prompt: str) -> str:
        url = f"{self.base_url}/chat/completions"
        headers = {"Content-Type": "application/json"}
        payload = {
            "model": self.model,
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            "temperature": 0.3,
        }
        resp = requests.post(url, headers=headers, data=json.dumps(payload))
        resp.raise_for_status()
        data = resp.json()
        return data["choices"][0]["message"]["content"]


lm_client = LMStudioClient()

<!-- ## 5. ES 인덱스 & Qdrant 컬렉션 생성 + 더미 논문 데이터 적재

실제로는 여기에 논문 메타데이터/초록/본문을 크롤링/전처리해서 넣으면 된다.  
지금은 **3개의 더미 논문**만 사용해서 플로우를 보여준다. -->


In [21]:
import elasticsearch
import elastic_transport

print("elasticsearch 버전:", elasticsearch.__version__)
print("elastic-transport 버전:", elastic_transport.__version__)


elasticsearch 버전: (8, 14, 0)
elastic-transport 버전: 8.17.1


In [22]:
# from elasticsearch import Elasticsearch

# es = Elasticsearch("http://localhost:9200")

# print(es.info())
 

In [23]:
INDEX_NAME = "papers_index"
VECTOR_COLLECTION = "papers_abstract"
CHUNK_COLLECTION = "papers_chunks"


def reset_es_index():
    if es.indices.exists(index=INDEX_NAME):
        es.indices.delete(index=INDEX_NAME)
    es.indices.create(
        index=INDEX_NAME,
        body={
            "mappings": {
                "properties": {
                    "title": {"type": "text"},
                    "abstract": {"type": "text"},
                    "year": {"type": "integer"},
                    "venue": {"type": "keyword"},
                    "citations": {"type": "integer"},
                    "url": {"type": "keyword"},
                }
            }
        },
    )
    print("[ES] index reset 완료")



In [24]:
def reset_qdrant_collections():
    # Abstract용 컬렉션
    existing = [c.name for c in qdrant.get_collections().collections]
    if VECTOR_COLLECTION in existing:
        qdrant.delete_collection(VECTOR_COLLECTION)
    qdrant.recreate_collection(
        collection_name=VECTOR_COLLECTION,
        vectors_config=VectorParams(size=embedder.dim, distance=Distance.COSINE),
    )
    # Chunk용 컬렉션
    existing = [c.name for c in qdrant.get_collections().collections]
    if CHUNK_COLLECTION in existing:
        qdrant.delete_collection(CHUNK_COLLECTION)
    qdrant.recreate_collection(
        collection_name=CHUNK_COLLECTION,
        vectors_config=VectorParams(size=embedder.dim, distance=Distance.COSINE),
    )
    print("[Qdrant] collections reset 완료")

In [25]:
# 더미 논문 데이터
dummy_papers = [
    Paper(
        paper_id="p1",
        title="Agentic RAG for Scientific Paper Search",
        abstract="We propose an agentic retrieval-augmented generation framework for scientific paper search ...",
        year=2024,
        venue="NeurIPS",
        citations=50,
        url="https://example.com/p1",
    ),
    Paper(
        paper_id="p2",
        title="Hybrid BM25 and Dense Retrieval for Academic Recommendation",
        abstract="This work combines sparse BM25 retrieval and dense embedding-based search for academic recommendations ...",
        year=2023,
        venue="ICLR",
        citations=120,
        url="https://example.com/p2",
    ),
    Paper(
        paper_id="p3",
        title="Self-Attention is All You Need",
        abstract="We introduce the Transformer architecture, based solely on self-attention mechanisms ...",
        year=2017,
        venue="NIPS",
        citations=10000,
        url="https://example.com/p3",
    ),
]

In [26]:



def ingest_dummy_data():
    # ES에 인덱싱
    actions = []
    for p in dummy_papers:
        actions.append({
            "_index": INDEX_NAME,
            "_id": p.paper_id,
            "_source": {
                "title": p.title,
                "abstract": p.abstract,
                "year": p.year,
                "venue": p.venue,
                "citations": p.citations,
                "url": p.url,
            },
        })
    helpers.bulk(es, actions)
    print(f"[ES] {len(actions)}개 문서 인덱싱 완료")

    # Qdrant: abstract 벡터
    abstracts = [p.abstract for p in dummy_papers]
    vecs = embedder.encode(abstracts)
    points = []
    for i, p in enumerate(dummy_papers):
        points.append(
            PointStruct(
                id=i + 1,
                vector=vecs[i].tolist(),
                payload={
                    "paper_id": p.paper_id,
                    "title": p.title,
                    "abstract": p.abstract,
                    "year": p.year,
                    "venue": p.venue,
                    "citations": p.citations,
                    "url": p.url,
                },
            )
        )
    qdrant.upsert(collection_name=VECTOR_COLLECTION, points=points)
    print(f"[Qdrant] abstract vectors {len(points)}개 업서트 완료")

    # Qdrant: chunk 벡터 (토이: abstract를 chunk처럼 재사용)
    chunk_points = []
    cid = 1
    for p in dummy_papers:
        text = f"Core idea of {p.title}: {p.abstract}"
        vec = embedder.encode([text])[0]
        chunk_points.append(
            PointStruct(
                id=cid,
                vector=vec.tolist(),
                payload={
                    "paper_id": p.paper_id,
                    "section": "Intro",
                    "chunk_id": f"{p.paper_id}_c1",
                    "text": text,
                },
            )
        )
        cid += 1
    qdrant.upsert(collection_name=CHUNK_COLLECTION, points=chunk_points)
    print(f"[Qdrant] chunk vectors {len(chunk_points)}개 업서트 완료")




<!-- ## 6. 에이전트 함수 정의 (발화 분류, 전략, 검색, 병합, chunk, 답변) -->

In [27]:
def classify_utterance_agent(state: AgentState) -> AgentState:
    text = state["query_text"].strip()

    if "논문" in text and ("원본" in text or "보여줘" in text or "링크" in text):
        utter = UtteranceType.SPECIFIC_PAPER
    else:
        tokens = text.split()
        if len(tokens) <= 5 and any(k in text.lower() for k in ["iclr", "neurips", "icml", "acl", "2023", "2024", "2025"]):
            utter = UtteranceType.KEYWORD_TOPIC
        else:
            utter = UtteranceType.NL_TOPIC

    state["utterance_type"] = utter.value
    print(f"[utterance] 분류 결과: {utter.value}")
    return state


def strategy_agent(state: AgentState) -> AgentState:
    utter = UtteranceType(state["utterance_type"])
    strategy = SEARCH_STRATEGY_MAP[utter]
    state["strategy"] = strategy
    print(f"[strategy] 전략 선택: {strategy}")
    return state

In [28]:
# 실제 실행
reset_es_index()


[ES] index reset 완료


In [29]:
reset_qdrant_collections()

/var/folders/1l/ry5k1k8n3cb61x096k96f2fm0000gn/T/ipykernel_3280/4088941107.py:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(
/var/folders/1l/ry5k1k8n3cb61x096k96f2fm0000gn/T/ipykernel_3280/4088941107.py:14: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


[Qdrant] collections reset 완료


In [30]:

ingest_dummy_data()

[ES] 3개 문서 인덱싱 완료
[Qdrant] abstract vectors 3개 업서트 완료
[Qdrant] chunk vectors 3개 업서트 완료


In [31]:
def keyword_search_agent(state: AgentState) -> AgentState:
    strategy = state["strategy"]
    if not strategy.use_keyword_search:
        return state

    q = state["query_text"]
    # ES multi_match
    body = {
        "query": {
            "multi_match": {
                "query": q,
                "fields": ["title^4", "abstract^2", "venue^1"],
                "fuzziness": "AUTO",
            }
        },
        "size": 20,
    }
    resp = es.search(index=INDEX_NAME, body=body)
    hits: List[PaperHit] = []
    for h in resp["hits"]["hits"]:
        s = h["_source"]
        paper = Paper(
            paper_id=h["_id"],
            title=s.get("title", ""),
            abstract=s.get("abstract", ""),
            year=s.get("year"),
            venue=s.get("venue"),
            citations=s.get("citations"),
            url=s.get("url"),
        )
        hits.append(PaperHit(paper=paper, score=h["_score"]))

    hits.sort(key=lambda x: x.score, reverse=True)
    state["keyword_hits"] = hits
    print(f"[keyword] {len(hits)}개 후보 발견")
    return state

In [32]:
def semantic_search_agent(state: AgentState) -> AgentState:
    strategy = state["strategy"]
    if not strategy.use_semantic_abstract_search:
        return state

    q_vec = embedder.encode([state["query_text"]])[0]

    results = qdrant.search(
        collection_name=VECTOR_COLLECTION,
        query_vector=q_vec.tolist(),
        limit=20,
    )
    hits: List[PaperHit] = []
    for r in results:
        p = r.payload
        paper = Paper(
            paper_id=p["paper_id"],
            title=p["title"],
            abstract=p["abstract"],
            year=p.get("year"),
            venue=p.get("venue"),
            citations=p.get("citations"),
            url=p.get("url"),
        )
        hits.append(PaperHit(paper=paper, score=r.score))

    hits.sort(key=lambda x: x.score, reverse=True)
    state["semantic_hits"] = hits
    print(f"[semantic] {len(hits)}개 후보 발견")
    return state

In [33]:
def merge_and_select_agent(current_year: int = 2025):
    def _agent(state: AgentState) -> AgentState:
        strategy = state["strategy"]
        kw_hits = state.get("keyword_hits", [])
        sem_hits = state.get("semantic_hits", [])

        scores: Dict[str, float] = {}
        meta: Dict[str, PaperHit] = {}

        def add_scores(hits: List[PaperHit], weight: float, k_rrf: int = 60):
            for rank, h in enumerate(hits, start=1):
                pid = h.paper.paper_id
                scores.setdefault(pid, 0.0)
                scores[pid] += weight * (1.0 / (k_rrf + rank))
                meta.setdefault(pid, h)

        add_scores(kw_hits, strategy.keyword_weight)
        add_scores(sem_hits, strategy.semantic_weight)

        merged: List[PaperHit] = []
        for pid, base in scores.items():
            p = meta[pid].paper
            recency = 0.0
            if p.year:
                delta = max(current_year - p.year, 0)
                recency = 1 / (1 + delta)
            cit = p.citations or 0
            cit_score = math.log1p(cit) / 10.0

            final = (
                strategy.relevance_weight * base
                + strategy.recency_weight * recency
                + strategy.citation_weight * cit_score
            )
            merged.append(PaperHit(paper=p, score=final))

        merged.sort(key=lambda x: x.score, reverse=True)
        state["merged_hits"] = merged

        utter = UtteranceType(state["utterance_type"])
        top_n = 3 if utter == UtteranceType.SPECIFIC_PAPER else 5
        state["top_papers"] = merged[:top_n]
        print(f"[merge] top {top_n} 선택: {[h.paper.title for h in state['top_papers']]}")
        return state

    return _agent

In [40]:
def chunk_retrieval_agent(state: AgentState) -> AgentState:
    strategy = state["strategy"]
    if not strategy.use_chunk_search:
        return state

    top_papers = state.get("top_papers", [])
    if not top_papers:
        return state

    paper_ids = [h.paper.paper_id for h in top_papers]

    # Qdrant filter로 paper_id in paper_ids
    # 단순 search + 이후 필터링 (토이 버전)
    q_vec = embedder.encode([state["query_text"]])[0]
    results = qdrant.search(
        collection_name=CHUNK_COLLECTION,
        query_vector=q_vec.tolist(),
        limit=20,
    )
    chunks: List[ChunkHit] = []
    for r in results:
        p = r.payload
        if p["paper_id"] not in paper_ids:
            continue
        chunks.append(
            ChunkHit(
                paper_id=p["paper_id"],
                section=p.get("section"),
                chunk_id=p["chunk_id"],
                text=p["text"],
                score=r.score,
            )
        )

    chunks.sort(key=lambda x: x.score, reverse=True)
    state["chunks"] = chunks[:10]
    print(f"[chunk] {len(state['chunks'])}개 chunk 선택")
    return state

In [35]:
def answer_agent(state: AgentState) -> AgentState:
    strategy = state["strategy"]
    if not strategy.use_chunk_search:
        # 논문 리스트만 반환하는 전략
        lines = ["이 질의는 요약 대신 관련 논문 리스트를 우선 제공하는 전략입니다."]
        for h in state.get("top_papers", []):
            lines.append(f"- {h.paper.title} ({h.paper.year}, {h.paper.venue})")
        state["answer"] = "\n".join(lines)
        print("[answer] chunk 미사용 전략 → 리스트만 생성")
        return state

    top_papers = state.get("top_papers", [])
    chunks = state.get("chunks", [])
    if not top_papers:
        state["answer"] = "관련 논문을 찾지 못했습니다."
        print("[answer] top_papers 없음")
        return state

    paper_list_str = "\n".join(
        f"- {h.paper.title} ({h.paper.year}, {h.paper.venue}) citations≈{h.paper.citations}"
        for h in top_papers
    )
    context_str = "\n\n".join(
        f"[{ch.paper_id} - {ch.section}]\n{ch.text}" for ch in chunks
    )

    system_prompt = (
        "너는 논문 검색 및 요약을 도와주는 어시스턴트야. "
        "사용자의 질문과 관련 논문 본문/메타 정보를 보고, "
        "근거를 바탕으로 한국어로 명확하게 요약과 추천을 제공해."
    )

    user_prompt = f"""사용자 질문:
{state["query_text"]}

관련 논문 목록:
{paper_list_str}

논문 본문 발췌:
{context_str}

요구사항:
1. 질문에 대해 한 단락으로 핵심 요약을 먼저 작성해줘.
2. 이어서 각 논문이 이 주제에서 어떤 기여를 했는지 bullet로 정리해줘.
3. 마지막에 '추천 논문' 섹션을 만들어, 특히 먼저 읽으면 좋은 논문 1~2편을 골라 이유와 함께 제안해줘.
"""

    lm: LMStudioClient = state["lm_client"]
    answer = lm.chat(system_prompt=system_prompt, user_prompt=user_prompt)
    state["answer"] = answer
    print("[answer] LLM 요약 생성 완료")
    return state

<!-- ## 7. LangGraph 그래프 구성 및 실행 -->

In [36]:
def build_graph(current_year: int = 2025):
    graph = StateGraph(AgentState)

    graph.add_node("classify_utterance", classify_utterance_agent)
    graph.add_node("strategy", strategy_agent)
    graph.add_node("keyword_search", keyword_search_agent)
    graph.add_node("semantic_search", semantic_search_agent)
    graph.add_node("merge_and_select", merge_and_select_agent(current_year=current_year))
    graph.add_node("chunk_retrieval", chunk_retrieval_agent)
    graph.add_node("answer", answer_agent)

    graph.set_entry_point("classify_utterance")
    graph.add_edge("classify_utterance", "strategy")
    graph.add_edge("strategy", "keyword_search")
    graph.add_edge("keyword_search", "semantic_search")
    graph.add_edge("semantic_search", "merge_and_select")
    graph.add_edge("merge_and_select", "chunk_retrieval")
    graph.add_edge("chunk_retrieval", "answer")
    graph.add_edge("answer", END)

    app = graph.compile()
    return app


graph_app = build_graph()

<!-- ## 8. 예시 질의 실행

아래 예시 질의를 돌려보면서:

- 발화 분류 결과
- 전략 선택
- ES / Qdrant 검색
- 병합/랭킹
- chunk 선택
- Qwen3(LM Studio) 기반 최종 답변

까지 흐름을 확인할 수 있다. -->


In [38]:
examples = [
    "agentic rag 논문 추천",
    "프롬프트 성능 향상에 대한 좋은 논문을 추천해줘",
    "self-attention 관련 논문 알려줘",
]

for q in examples:
    print("\n" + "=" * 80)
    print("질문:", q)
    init_state: AgentState = {
        "query_text": q,
        "lm_client": lm_client,
    }
    final_state = graph_app.invoke(init_state)
    print("\n[최종 논문 리스트]")
    for h in final_state.get("top_papers", []):
        print("-", h.paper.title, h.paper.year, h.paper.venue, h.paper.url)
    print("\n[최종 answer]\n")
    print(final_state.get("answer"))



질문: agentic rag 논문 추천
[utterance] 분류 결과: natural_language_topic
[strategy] 전략 선택: SearchStrategy(utterance_type=<UtteranceType.NL_TOPIC: 'natural_language_topic'>, use_keyword_search=True, use_semantic_abstract_search=True, use_chunk_search=True, keyword_weight=0.4, semantic_weight=0.6, relevance_weight=0.7, recency_weight=0.15, citation_weight=0.15)
[keyword] 1개 후보 발견


/var/folders/1l/ry5k1k8n3cb61x096k96f2fm0000gn/T/ipykernel_3280/1669883693.py:8: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(


[semantic] 3개 후보 발견
[merge] top 5 선택: ['Self-Attention is All You Need', 'Agentic RAG for Scientific Paper Search', 'Hybrid BM25 and Dense Retrieval for Academic Recommendation']


/var/folders/1l/ry5k1k8n3cb61x096k96f2fm0000gn/T/ipykernel_3280/641953078.py:15: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(


[chunk] 3개 chunk 선택
[answer] LLM 요약 생성 완료

[최종 논문 리스트]
- Self-Attention is All You Need 2017 NIPS https://example.com/p3
- Agentic RAG for Scientific Paper Search 2024 NeurIPS https://example.com/p1
- Hybrid BM25 and Dense Retrieval for Academic Recommendation 2023 ICLR https://example.com/p2

[최종 answer]

“Agentic RAG”는 RAG(Retrieval-Augmented Generation)을 기반으로 한 시스템에서 ‘주체(агент)’가 스스로 검색 및 생성 과정을 판단하고 실행하는 방식을 의미하며, 특히 과학 논문 검색 등 복잡한 탐색 작업에 적용된 최신 연구입니다. 이는 단순히 문서를 검색하고 읽는 것 이상의 ‘행동 기반’ 인공지능 시스템을 의미합니다. 관련 논문 중 2024년 NeurIPS에서 발표된 “Agentic RAG for Scientific Paper Search”는 이 주제의 최신 핵심 연구로, 2023년 ICLR의 “Hybrid BM25 and Dense Retrieval for Academic Recommendation”은 RAG의 전통적 접근법을 보완하는 방식으로, 2017년 NIPS의 “Self-Attention is All You Need”은 RAG의 기초 기술인 변형된 어텐션 기반 모델링을 제공하는 논문입니다.

- **Agentic RAG for Scientific Paper Search (2024, NeurIPS)**: 주체 기반의 RAG 시스템을 제안하여, 과학 논문 검색 시 검색 결과를 평가하고, 적절한 질문을 재검색하거나 생성을 재조정하는 능력을 가진 ‘주체’ 역할을 수행함으로써, 사용자의 의도를 더 정확히 이해하고 응답을 생성함.
- **Hybrid BM25 and Dense R

In [39]:
import requests, json

resp = requests.post(

    "http://localhost:1234/v1/chat/completions",  # 포트는 LM Studio에 맞춰 수정
    headers={"Content-Type": "application/json"},
    data=json.dumps({
        "model": "여기에 LM Studio에 뜨는 모델 이름",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "hi"},
        ],
    }),
)
print(resp.status_code)
print(resp.text[:500])


200
{
  "id": "chatcmpl-6ueydxw5j3lugb9o237g5s",
  "object": "chat.completion",
  "created": 1763389768,
  "model": "qwen/qwen3-vl-4b",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today? 😊",
        "tool_calls": []
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 20,
    "completion_tokens": 12,
    "total_tokens": 32
  },
  "stats": {},
  "system_fin


In [25]:
# 